<a href="https://colab.research.google.com/github/sameep114/myproject/blob/master/summarization_starting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

#get gpu deviece name
device_name=tf.test.gpu_device_name()

#the device name should look likefollowing
if device_name=='/device:GPU:0':
  print("found GPU at:{}",format(device_name))
else:
  raise SystemError('Gpu Device Not FOund')

found GPU at:{} /device:GPU:0


In [2]:
import torch

# if there is GPU available
if torch.cuda.is_available():

  #requesting Pytorch tof use GPU
  device= torch.device("cuda")
  print("there is %d GPU(s)  available." % torch.cuda.device_count())
  print("we will use the gpu:", torch.cuda.get_device_name(0))

#if not...
else:
  print('NO GPU Available using cpu instead.')
  device=torch.device('Cpu')


there is 1 GPU(s)  available.
we will use the gpu: Tesla T4


installing hugging face library

In [3]:
!pip install transformers

     |████████████████████████████████| 1.3MB 7.3MB/s 
     |████████████████████████████████| 890kB 25.7MB/s 
     |████████████████████████████████| 2.9MB 36.1MB/s 
     |████████████████████████████████| 1.1MB 50.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=a4bbeca7f53ed965a1a994203c300fd38c3800772890a4225b8c2521ac922ca0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os

import pandas as pd




TOKENIZATION

In [ ]:
print("we are now loading our dataset...")
df=pd.read_csv("/content/drive/MyDrive/Datasets/nepali_news_dataset_20_categories_large/Agriculture/1.txt",delimiter='\n' ,header=None,names=['sentences'])



we are now loading our dataset...


In [ ]:
#report number of sentences.
print("number of training sentences: {:,}\n".format(df.shape[0]))

# displaying random rows
df.head(10)

number of training sentences: 11



,sentences
0,काठमाडौंमा पहिलो पटक स्ट्रबेरीको व्यवसायिक खेत...
1,यसअघि स्ट्रबेरी नुवाकोटमा गरिँदै आएको थियो। अध...
2,बीस–पच्चीस वर्षअघि ककनीमा जाइटी भन्ने संस्थाले...
3,‘मैले नुवाकोटमा ग्रीन हाउस बनाउने थोपा सिँचाइक...
4,अधिकारीको फर्ममा अहिले स्ट्रबेरी फल्न सुरु गरे...
5,५ देखि २५ डिग्री तापक्रममा स्ट्रबेरी लगाउन सकि...
6,विभिन्न शुभ कार्यहरुमा समेत स्ट्रबेरीलाई प्याक...
7,‘नेपालमा पनि स्ट्रबेरीको उत्पादन गर्न सके बजार...
8,स्ट्रबेरीको जाम विश्वमै चर्चित रहेको उनले बताए...
9,विदेशमा लामो समयसम्म काम गरेकोले पनि आफूले स्व...


In [ ]:
#get list of sentences
sentences=df.sentences.values

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

#load bert tokenizer
print("loading BERT  tokenizer...")
tokenizer=BertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower=True)

loading BERT  tokenizer...


In [ ]:
print('original:',sentences[0])

# print paragrapg splitted into tokens.
print('Tokenized:',tokenizer.tokenize(sentences[0]))

#print sentences mapped into token ids
print('Token IDs:',tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

original: काठमाडौंमा पहिलो पटक स्ट्रबेरीको व्यवसायिक खेती सुरु भएको छ । एक दशकसम्म इजरायली कृषि फार्ममा बसेर काम गरेका श्रीकृष्ण अधिकारीले तारकेश्वर नगरपालिका ४ गोलढुंगामा उत्पादन सुरु गरेका हुन्।
Tokenized: ['का', '##ठ', '##मा', '##ड', '##ौ', '##ं', '##मा', 'पहिलो', 'प', '##टक', 'स', '##्ट', '##्र', '##बे', '##री', '##को', 'व', '##्य', '##व', '##सा', '##यिक', 'ख', '##ेत', '##ी', 'स', '##ुरु', 'भएको', 'छ', '।', 'एक', 'दशक', '##सम', '##्म', 'इ', '##ज', '##रा', '##य', '##ली', 'क', '##ृ', '##षि', 'फ', '##ार', '##्म', '##मा', 'ब', '##से', '##र', 'काम', 'गरेका', 'श्री', '##क', '##ृ', '##ष', '##्ण', 'अधिकार', '##ील', '##े', 'त', '##ार', '##के', '##श', '##्वर', 'नगर', '##पालिका', '४', 'ग', '##ोल', '##ढ', '##ु', '##ंगा', '##मा', 'उत्पादन', 'स', '##ुरु', 'गरेका', 'हुन्', '।']
Token IDs: [11081, 35247, 12347, 20691, 78530, 14018, 12347, 92770, 885, 76826, 898, 25695, 18321, 85002, 20161, 12512, 895, 14251, 15070, 35127, 96823, 866, 53316, 10914, 898, 56902, 31454, 871, 920, 11186, 103202, 105794

In [ ]:
# tokenize all of the sentences and map the tokens to their respective word ids
input_ids=[]

#for every sentences
for sent in sentences:
  encodded_sent=tokenizer.encode(
                    sent, add_special_tokens=True,
                        )
input_ids.append(encodded_sent)

# print first paragraph now as list of ids.
print('original:',sentences[0])
print('token IDs',input_ids[0])

original: काठमाडौंमा पहिलो पटक स्ट्रबेरीको व्यवसायिक खेती सुरु भएको छ । एक दशकसम्म इजरायली कृषि फार्ममा बसेर काम गरेका श्रीकृष्ण अधिकारीले तारकेश्वर नगरपालिका ४ गोलढुंगामा उत्पादन सुरु गरेका हुन्।
token IDs [101, 867, 50346, 11453, 56523, 111194, 13432, 60701, 76131, 881, 65430, 11208, 898, 12878, 28462, 22078, 15801, 32629, 11549, 75315, 11845, 852, 28863, 63401, 11554, 40107, 15399, 26472, 876, 11208, 111194, 28462, 49050, 14070, 12347, 32824, 87291, 866, 53316, 10914, 44069, 47313, 37920, 867, 31277, 43583, 70103, 10914, 12512, 888, 16380, 34231, 871, 920, 29953, 12347, 92770, 18187, 76826, 881, 65430, 11208, 898, 103435, 22078, 84237, 30472, 895, 14251, 15070, 35127, 96823, 72954, 12347, 852, 28863, 18438, 13665, 30693, 66927, 64645, 882, 36471, 10914, 79263, 920, 102]


In [ ]:
# before padding we need to know which is the longest sentence in our dataset 
print('max length of sentence',max([len(sen) for sen in input_ids ]))

max length of sentence 87


In [ ]:
# lets use pad_sequencecs utility function in order to do this work
from keras.preprocessing.sequence import pad_sequences
# lets choose Max_len == 128 and apply the padding
MAX_LEN=128

print('\n padding/truncating all sentences to %d values....'% MAX_LEN)

print("\n padding token :'{:}', Id :{:}".format(tokenizer.pad_token, tokenizer.pad_token_id))

#pad our input tokens with valie 0.
input_ids=pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long",
                         value=0, truncating="post",padding="post")

print("\n Done")


 padding/truncating all sentences to 128 values....

 padding token :'[PAD]', Id :0

 Done


attention masks


In [ ]:
#create attention masks
attention_masks= []
#for each sentence
for sent in input_ids:
  att_mask=[int(token_id > 0) for token_id in sent] #reason: token id> 0 real token, token id== 0 padding
  attention_masks.append(att_mask)

TRAINING AND VALIDATION SPLIT


In [ ]:
#training
from sklearn.model_selection import train_test_split

#use 90% for training and 10 percent for validation
train_inputs, validation_inputs, train_labels, validation_labels

In [20]:
from transformers import  T5Tokenizer, TFT5ForConditionalGeneration
model=BertForConditionalGeneration.from_pretrained('t5-small')
tokenizer=tokenizer = T5Tokenizer.from_pretrained('t5-small')

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [24]:
from transformers import BartTokenizer, BartForConditionalGeneration
model=BartForConditionalGeneration.from_pretrained('facebook/bart-large-mnli')
tokenizer= BartTokenizer.from_pretrained('facebook/bart-large-mnli')

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForConditionalGeneration: ['classification_head.dense.weight', 'classification_head.dense.bias', 'classification_head.out_proj.weight', 'classification_head.out_proj.bias', 'model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
text=sentences.strip().replace("\n","")
preprocessed_text="summarize: "+Text
tokens_input=tokenizer.encode(Preprocessed_text,return_tensors='tf')

AttributeError: ignored

In [ ]:
print('original:',sentences[0])

# print paragrapg splitted into tokens.
print('Tokenized:',tokenizer.tokenize(sentences[0]))

#print sentences mapped into token ids
print('Token IDs:',tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

original: काठमाडौंमा पहिलो पटक स्ट्रबेरीको व्यवसायिक खेती सुरु भएको छ । एक दशकसम्म इजरायली कृषि फार्ममा बसेर काम गरेका श्रीकृष्ण अधिकारीले तारकेश्वर नगरपालिका ४ गोलढुंगामा उत्पादन सुरु गरेका हुन्।
Tokenized: ['▁', 'काठमाडौंमा', '▁', 'पहिलो', '▁', 'पटक', '▁', 'स्ट्रबेरीको', '▁', 'व्यवसायिक', '▁', 'खेती', '▁', 'सुरु', '▁', 'भएको', '▁', 'छ', '▁', '।', '▁', 'एक', '▁', 'दशकसम्म', '▁', 'इजरायली', '▁', 'कृषि', '▁', 'फार्ममा', '▁', 'बसेर', '▁', 'काम', '▁', 'गरेका', '▁', 'श्रीकृष्ण', '▁', 'अधिकारीले', '▁', 'तारकेश्वर', '▁', 'नगरपालिका', '▁', '४', '▁', 'गोलढुंगामा', '▁', 'उत्पादन', '▁', 'सुरु', '▁', 'गरेका', '▁', 'हुन्।']
Token IDs: [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2]


In [ ]:
# tokenize all of the sentences and map the tokens to their respective word ids
input_ids=[]

#for every sentences
for sent in sentences:
  encodded_sent=tokenizer.encode(
                    sent, add_special_tokens=True,
                        )
input_ids.append(encodded_sent)

# print first paragraph now as list of ids.
print('original:',sentences[0])
print('token IDs',input_ids[0])

original: काठमाडौंमा पहिलो पटक स्ट्रबेरीको व्यवसायिक खेती सुरु भएको छ । एक दशकसम्म इजरायली कृषि फार्ममा बसेर काम गरेका श्रीकृष्ण अधिकारीले तारकेश्वर नगरपालिका ४ गोलढुंगामा उत्पादन सुरु गरेका हुन्।
token IDs [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 1]


In [18]:
import numpy as np

input_ids=tokenizer.tokenize(text)
print(input_ids)

['▁ma', '▁I', '▁ka', 'ile', '▁has', 'eko', '▁th', 'iye', '▁mero', '▁samu', '▁', 'timi', '▁', 'ruda', '▁kina', '▁te', 'sai', '▁gali', '▁gar', 'xe', 'u', '▁badh', 'yata', '▁ma', '▁ta', 'dha', '▁hud', 'a', 'a', '▁saba', 'i', '▁th', 'aha', '▁xa', '▁am', '▁', 'Sorry', '▁hi', '▁there', '▁', 'timi', 'lai', '▁kasar', 'i', '▁ma', '▁ta', 'dha', '▁b', 'haye', '▁dhe', 'rai', '▁my', '▁gar', 'the', '▁tim', 'li', '▁kina', '▁aja', '▁na', 'ulo', '▁bane', '▁yo', '▁ma', '▁kaha', '▁khus', 'i', '▁xa', '▁', 'timi', 'lai', '▁', 'arul', 'e', '▁xu', 'da', '▁kina', '▁te', 'sai', '▁gali', '▁gar', 'xe', 'u', '▁badh', 'yata', '▁ta', 'dha', '▁hud', 'a']


In [ ]:
input_tokens=np.array(input_ids)

In [ ]:
summary_ids=model.generate(input_tokens, 
                           max_length=50)
summary=tokenizer.decode(summary_ids)

AttributeError: ignored

In [ ]:
preprocess_text = sentences.strip().replace("\n","")
t5_prepared_Text = "summarize: "+preprocess_text
print ("original text preprocessed: \n", preprocess_text)

tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt")


original text preprocessed: 
 सहज रुपमा भन्नुपर्दा नेपाली भनेको नेपालमा बस्ने नागरिक हो । हामी नेपाली नेपाललाई अंग्रेजबाट मुक्त गराएर स्वच्छ बनाउने बिर योद्धा हो । नेपाल अनेक जात, अनेक धर्म र अनेक भाषाभेष मिसिएर बनेको छ। हामी नेपालीमा घमण्ड र स्वार्थ भन्ने भावना उठेको छैन। हामी निस्वार्थी र धर्मको बाटोमा लाग्ने मानिस हौ। हामी रिस,राग,कपट र छल कहिले गर्दैनौ। हाम्रो देश स्वर्ग झैँ सुन्दर र चद्रमा झैँ चम्किलो छ। हामी नेपाली एकतामा बसेर हरेक कम गर्छौ र सफलता प्राप्त गर्छौ।                                            हाम्रो देशमा एक पछि एक महान पुरुषहरु जन्मेका हुन्। जस्तै-प्रिथिवी नारायण शाह,बलभद्र कुँवर आदि। उहाँहरुले ठुला ठुला काम गरी जानुभएको छ। उहाँहरुले देशका लागि केही न केही योगदान दिएर गएका हन्। हामी नेपाली तिनीहरुका छोराछोरी हौ र त्यसै कारणले हामीले पनि देशको लागि केही गर्नुपर्छ। हाम्रो देशमा बिर गोर्खालीहरु जन्मेका हुन्, उनिहेरुले शत्रुहरुलाई पराजित गरेर हाम्रो देशलाई जोगाएका हुन्। उनीहरु नेपालमै जन्मेका थिए। उनीहरुको जस्तै हामीले पनि आफ्नो प्राणको चिन्ता नगरी देशलाई हरेक परिस्थिति

In [ ]:
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)

AttributeError: ignored

In [25]:
text="""
सहज रुपमा भन्नुपर्दा नेपाली भनेको नेपालमा बस्ने नागरिक हो । हामी नेपाली नेपाललाई अंग्रेजबाट मुक्त गराएर स्वच्छ बनाउने बिर योद्धा हो । नेपाल अनेक जात, अनेक धर्म र अनेक भाषाभेष मिसिएर बनेको छ। हामी नेपालीमा घमण्ड र स्वार्थ भन्ने भावना उठेको छैन। हामी निस्वार्थी र धर्मको बाटोमा लाग्ने मानिस हौ। हामी रिस,राग,कपट र छल कहिले गर्दैनौ। हाम्रो देश स्वर्ग झैँ सुन्दर र चद्रमा झैँ चम्किलो छ। हामी नेपाली एकतामा बसेर हरेक कम गर्छौ र सफलता प्राप्त गर्छौ।
                                            हाम्रो देशमा एक पछि एक महान पुरुषहरु जन्मेका हुन्। जस्तै-प्रिथिवी नारायण शाह,बलभद्र कुँवर आदि। उहाँहरुले ठुला ठुला काम गरी जानुभएको छ। उहाँहरुले देशका लागि केही न केही योगदान दिएर गएका हन्। हामी नेपाली तिनीहरुका छोराछोरी हौ र त्यसै कारणले हामीले पनि देशको लागि केही गर्नुपर्छ। हाम्रो देशमा बिर गोर्खालीहरु जन्मेका हुन्, उनिहेरुले शत्रुहरुलाई पराजित गरेर हाम्रो देशलाई जोगाएका हुन्। उनीहरु नेपालमै जन्मेका थिए। उनीहरुको जस्तै हामीले पनि आफ्नो प्राणको चिन्ता नगरी देशलाई हरेक परिस्थितिबाट बचाउनुपर्छ। हामी नेपालीले नेपाललाई विकास तर्फ लानु पर्छ। हामी नेपालीले कर्म मात्र गर्नुपर्छ, फलको उपेक्षा गर्नु हुँदैन। हामी नेपालले विर गोर्खाली रगतले हार मान्दैनौ र गौतम बुद्धको शिक्षालाई चारै तर्फ बाटछौ। हामी नेपालीले आफ्नो ज्ञानले सबैलाई पवित्र बनाउनुपर्छ। हामी नेपालीले दुखसुख बाटी हरेकलाई सहयोग गर्छौ र गरिबलाई खान,कपडा,घर आदि गरिदिन्छौ।हामी नेपाली दयालु छौ र परिश्रम गरी हरेक कम गर्छौ।
                                                  
 हामी नेपालीले एकतामा बसी देशमा परेको हरेक समस्यालाई समाधान गर्छौ। नेपाल आमालै पिर परेको बेला हामी अगाडि बढ्छौ र देशलाई संकटबाट मुक्त गर्छौ। हामीले आफ्नो देशलाई दियोको जस्तो सधैभरिको उज्यालो रक्छौ। त्यसैले हामीले आफ्नो कर्तव्यलाई सधै पुरा गर्नुपर्छ र देशलाई स्वर्ग झैँ सुन्दर बनाउन सक्नुपर्छ । हामी नेपाली अरुको भर पर्दैनौ। हामीले आफुले नै कर्म गरी आफ्नो घरपरिवारलाई पाल्छौ र देशलाई सुन्दर र स्वच्छ राखना प्रयास गर्छौ।

यदि तपाइलाई अझै चाहियो भने कृपया comment मा इमेल लेखेर दिनुहोला अथवा subscribe गर्नुहोस ।
"""

In [14]:
text=""" ma I kaile haseko thiye mero samu timi ruda kina tesai gali garxeu badhyata ma tadha hudaa sabai thaha xa am Sorry hi there timilai kasari ma tadha bhaye dherai my garthe timli kina aja naulo bane yo ma kaha khusi xa timilai arule xuda kina tesai gali garxeu badhyata tadha huda"""

In [24]:
# text=text.decode(encoding='UTF-8',errors='strict')

In [26]:
Text = text.strip().replace("\n","")
Preprocessed_text = "summarize: "+Text
tokens_input = tokenizer.encode(Preprocessed_text,return_tensors="tf")

Token indices sequence length is longer than the specified maximum sequence length for this model (2987 > 1024). Running this sequence through the model will result in indexing errors


In [28]:
summary_ids = model.generate(tokens_input,
                             min_length=30,
                             max_length=40,
                             )

summary = tokenizer.decode(summary_ids[0])

ValueError: ignored

In [23]:
print(text)
print("\n\n")
print(summary)

During school days, I have developed an interest in playing games, staging dramas, debating and other extra-curricular activities.

I am a good debater and orator, athlete, and sportsman and at the same time, I hold top positions in the class in the academic field. All the qualities of head and heart have earned me a profound love and respect from my teachers and friends. My teachers encourage and help me in all possible ways. I am in the good books of all the teachers as well as the principal because I have won many medals, trophies, shields, and certificates for my extraordinary display of abilities in examinations, athletics, debates, and theatre.

The good ideas-like love for the motherland, devotion to duty, obedience- towards elders, service to the nation, helping the poor and the needy, nursing the sick, feeding the hungry, etc – are inculcated in the students during their school days.

Broadly speaking, school life is not only for learning, reading books or playing, but also a 

In [32]:
new_summary=summary.encode(encoding='UTF-8',errors='strict')

In [34]:
print(new_summary)

b'gew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb6hngew\xc3\xb

In [35]:
again_summary=summary.decode(encoding='UTF-8',errors='strict')

AttributeError: ignored

In [19]:
text="""During school days, I have developed an interest in playing games, staging dramas, debating and other extra-curricular activities.

I am a good debater and orator, athlete, and sportsman and at the same time, I hold top positions in the class in the academic field. All the qualities of head and heart have earned me a profound love and respect from my teachers and friends. My teachers encourage and help me in all possible ways. I am in the good books of all the teachers as well as the principal because I have won many medals, trophies, shields, and certificates for my extraordinary display of abilities in examinations, athletics, debates, and theatre.

The good ideas-like love for the motherland, devotion to duty, obedience- towards elders, service to the nation, helping the poor and the needy, nursing the sick, feeding the hungry, etc – are inculcated in the students during their school days.

Broadly speaking, school life is not only for learning, reading books or playing, but also a period during which all the good habits are acquired, bad habits are shunned and good conduct, fair play, and sound thinking are developed. Further, the healthy ideas of patriotism and nationalism are imbibed by the students during this period.

My school life shall prepare me for a sound and firm foundation upon which the building of my life is going to be erected. My mistakes and failures would guide me in my future life. I do not feel discouraged and disheartened because for me the failures are the stepping stones to success. My school life is a great and valuable experience, the best teacher. I shall always remember my school days fondly.
"""